<a href="https://colab.research.google.com/github/BhumikaAdhya05/Task_1/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"bhumikaadhya05","key":"76f7c91a508c639053f3ec8ca5a655a0"}'}

In [44]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [45]:
!pip install -q kaggle

In [46]:
!kaggle datasets download -d rounakbanik/the-movies-dataset

Dataset URL: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset
License(s): CC0-1.0
the-movies-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [47]:
!unzip the-movies-dataset.zip -d movies_dataset

Archive:  the-movies-dataset.zip
replace movies_dataset/credits.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace movies_dataset/keywords.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace movies_dataset/links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace movies_dataset/links_small.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace movies_dataset/movies_metadata.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace movies_dataset/ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace movies_dataset/ratings_small.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [48]:
import pandas as pd

movies = pd.read_csv('movies_dataset/movies_metadata.csv')
ratings = pd.read_csv('movies_dataset/ratings.csv')

<ipython-input-48-c1999d099e58>:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('movies_dataset/movies_metadata.csv')


In [49]:
import pandas as pd
import numpy as np

# Load data
movies = pd.read_csv('movies_dataset/movies_metadata.csv', low_memory=False)
ratings = pd.read_csv('movies_dataset/ratings.csv')

# Drop rows with bad data in movies
movies = movies.dropna(subset=['title', 'overview'])

# Convert ID columns to numeric
movies['id'] = pd.to_numeric(movies['id'], errors='coerce')
ratings = ratings.dropna()
ratings['movieId'] = pd.to_numeric(ratings['movieId'], errors='coerce')

# Drop invalid IDs
movies = movies.dropna(subset=['id'])
ratings = ratings.dropna(subset=['movieId'])

# Optional: reduce data size for quick prototyping
ratings = ratings.sample(frac=0.1, random_state=42)

<ipython-input-49-fb34d5b65a37>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['id'] = pd.to_numeric(movies['id'], errors='coerce')


In [50]:
# Fill NaNs with empty strings
movies['overview'] = movies['overview'].fillna('')

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create the TF-IDF matrix from overview
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['overview'])

# Check shape
print(tfidf_matrix.shape)

(44506, 75827)


In [52]:
from sklearn.neighbors import NearestNeighbors

# Fit NearestNeighbors model with cosine distance
nn = NearestNeighbors(n_neighbors=11, metric='cosine', algorithm='brute', n_jobs=-1)
nn.fit(tfidf_matrix)

# Find 10 nearest neighbors (excluding the movie itself)
distances, indices = nn.kneighbors(tfidf_matrix)

# Check the nearest neighbors of the first movie
print(indices[0])  # This shows the indices of the most similar movies

[    0 15282  2979 10271 24314 23644 28891 42566 37772 41887  8303]


In [53]:
# Use the full dataset
movies_full = movies.reset_index(drop=True)

# Create the reverse index again for the full dataset
indices = pd.Series(movies_full.index, index=movies_full['title'].str.lower()).drop_duplicates()

In [54]:
def get_recommendations(title, nn=nn, movies=movies_full, indices=indices):
    title = title.strip().lower()  # Normalize the input title
    # Check if the title exists in the dataset using case-insensitive matching
    match = indices.get(title)

    if not match:
        # Provide a list of close matches if the title is not found
        close_matches = [movie for movie in indices.index if title in movie.lower()]
        if not close_matches:
            return f"'{title}' not found in the database."
        return f"Did you mean: {', '.join(close_matches)}?"

    idx = indices[title]  # Get movie index based on title
    distances, indices_neigh = nn.kneighbors(tfidf_matrix[idx].reshape(1, -1))  # Ensure it's 2D

    movie_indices = indices_neigh[0][1:]  # Exclude the movie itself (which will be 0)

    return movies['title'].iloc[movie_indices]

In [55]:
# Test the recommendation system
recommended_movies = get_recommendations('The Matrix')
print(recommended_movies)

43254                        A Detective Story
43260                              Kid's Story
43256                             World Record
33363                                Algorithm
166                                    Hackers
20558    Underground: The Julian Assange Story
6491                                  Commando
23995                                 Who Am I
21917                           Berlin Express
9133                                  Takedown
Name: title, dtype: object


In [56]:
# Check column names in the dataset
print(movies_full.columns)

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


In [57]:
import ast

# Combine 'genres' and 'overview' columns safely
def get_genre_names(genre_str):
    try:
        genres_list = ast.literal_eval(genre_str)  # safer than eval()
        genre_names = [genre['name'] for genre in genres_list]
        return ' '.join(genre_names)
    except:
        return ''

# Apply function to genres
movies_full['genres_combined'] = movies_full['genres'].apply(get_genre_names)

# Now combine genres and overview
movies_full['combined_features'] = movies_full['genres_combined'] + ' ' + movies_full['overview'].fillna('')

# Preview
print(movies_full[['title', 'combined_features']].head())

                         title  \
0                    Toy Story   
1                      Jumanji   
2             Grumpier Old Men   
3            Waiting to Exhale   
4  Father of the Bride Part II   

                                   combined_features  
0  Animation Comedy Family Led by Woody, Andy's t...  
1  Adventure Fantasy Family When siblings Judy an...  
2  Romance Comedy A family wedding reignites the ...  
3  Comedy Drama Romance Cheated on, mistreated an...  
4  Comedy Just when George Banks has recovered fr...  


In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TF-IDF matrix
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf.fit_transform(movies_full['combined_features'])

In [59]:
from sklearn.neighbors import NearestNeighbors

# Fit NearestNeighbors model
nn = NearestNeighbors(n_neighbors=11, metric='cosine', algorithm='brute', n_jobs=-1)
nn.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=11)

In [60]:
import pandas as pd

# Build a Series to map movie titles to indices
indices = pd.Series(movies_full.index, index=movies_full['title'].str.lower()).drop_duplicates()

In [61]:
def get_recommendations(title, nn=nn, movies=movies_full, indices=indices):
    title = title.strip().lower()
    match = indices.get(title)

    if match is None:
        # Handle movie not found
        close_matches = [movie for movie in indices.index if title in movie.lower()]
        if not close_matches:
            return f"'{title}' not found in the database."
        return f"Did you mean: {', '.join(close_matches)}?"

    idx = match
    distances, indices_neigh = nn.kneighbors(tfidf_matrix[idx].reshape(1, -1))

    movie_indices = indices_neigh[0][1:]  # Skip the input movie itself
    return movies['title'].iloc[movie_indices]

In [62]:
recommended_movies = get_recommendations('Inception')
print(recommended_movies)

7436                                  Cypher
27984                          Spy, Stand Up
36936                         The Flying Man
13787    Transformers: Revenge of the Fallen
43861                               Altitude
41761                             Major Grom
36540                         Straight Story
23276     Mission: Impossible - Rogue Nation
10876                            Renaissance
13439                             Shadow Man
Name: title, dtype: object


In [63]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

# Load your ratings
movies = pd.read_csv('movies_dataset/movies_metadata.csv')
ratings = pd.read_csv('movies_dataset/ratings.csv')

# Create mappings
user_mapping = {id_: i for i, id_ in enumerate(ratings['userId'].unique())}
movie_mapping = {id_: i for i, id_ in enumerate(ratings['movieId'].unique())}

# Map user and movie IDs to indices
user_indices = ratings['userId'].map(user_mapping)
movie_indices = ratings['movieId'].map(movie_mapping)

# Build sparse matrix
rating_matrix_sparse = csr_matrix((ratings['rating'], (user_indices, movie_indices)))

print(f"Sparse Rating Matrix Shape: {rating_matrix_sparse.shape}")  # (num_users, num_movies)

<ipython-input-63-f2061e22627f>:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('movies_dataset/movies_metadata.csv')


Sparse Rating Matrix Shape: (270896, 45115)


In [64]:
from sklearn.neighbors import NearestNeighbors

# Initialize the KNN model for user-based collaborative filtering using cosine similarity
nn_model_sparse = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=11, n_jobs=-1)

# Fit the model using the sparse matrix
nn_model_sparse.fit(rating_matrix_sparse)

# Check if the model has been fitted
print(nn_model_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=11)


In [65]:
def recommend_movies(user_id, rating_matrix_sparse, nn_model, user_mapping, movie_mapping, movies_df, n_recommendations=10):
    # Map the user_id to user_index
    if user_id not in user_mapping:
        print(f"User {user_id} not found.")
        return []

    user_index = user_mapping[user_id]

    # Find nearest neighbors
    distances, indices = nn_model.kneighbors(rating_matrix_sparse[user_index], n_neighbors=20)

    print(f"User {user_id} nearest neighbors (indices): {indices}")
    print(f"User {user_id} nearest neighbors (distances): {distances}")

    # Get ratings from similar users
    similar_users_ratings = rating_matrix_sparse[indices.flatten()].toarray()

    # Calculate mean ratings across similar users
    mean_ratings = similar_users_ratings.mean(axis=0)

    # Find top rated unseen movies
    top_movie_indices = mean_ratings.argsort()[-n_recommendations:][::-1]

    # Reverse movie_mapping to map index -> movieId
    reverse_movie_mapping = {idx: id_ for id_, idx in movie_mapping.items()}
    recommended_movie_ids = [reverse_movie_mapping[idx] for idx in top_movie_indices if idx in reverse_movie_mapping]

    # Filter valid movie IDs
    recommended_movies = movies_df[movies_df['id'].astype(str).isin(map(str, recommended_movie_ids))]['title'].tolist()

    if not recommended_movies:
        print(f"No personalized recommendations found for User {user_id}. Falling back to popular movies.")
        recommended_movies = ['Inception', 'The Dark Knight', 'Avatar', 'The Avengers', 'Deadpool']

    return recommended_movies

In [66]:
recommended_movies = recommend_movies(
    user_id=6,
    rating_matrix_sparse=rating_matrix_sparse,
    nn_model=nn_model_sparse,
    user_mapping=user_mapping,
    movie_mapping=movie_mapping,
    movies_df=movies,
    n_recommendations=10
)

print(f"Recommended Movies for User 6: {recommended_movies}")

User 6 nearest neighbors (indices): [[     5  92837  93267  84911  39580 264231 111457 263109 118257 203434
   56942 269864  21340 101580 214770 162500  27683  21172  88035  28041]]
User 6 nearest neighbors (distances): [[0.         0.34905545 0.34905545 0.34905545 0.34905545 0.34905545
  0.34905545 0.34905545 0.34905545 0.34905545 0.34905545 0.34905545
  0.34905545 0.34905545 0.34905545 0.34905545 0.34905545 0.34905545
  0.34905545 0.34905545]]
Recommended Movies for User 6: ['Star Wars', 'Joe Versus the Volcano', 'Songwriter', 'Agenda Grinding America Down', 'Goblin']
